In [1]:
import cv2
import numpy as np
import IPython
import time
import tensorflow as tf
from IPython.display import Image

#weight_file = 'D:/YOLOTRAFFIC/archive/BX/yolov3_ts_train_7000.weights'
weight_file = 'D:/YOLOTRAFFIC/archive/backup/yolov3_ts_train_m_final.weights'
cfg_file = 'D:/YOLOTRAFFIC/archive/yolov3_ts_train.cfg'
name_file = 'D:/YOLOTRAFFIC/archive/classes.names'
weight_file2 = 'D:/YOLOTRAFFIC/archive/yolov4.weights'
cfg_file2 = 'D:/YOLOTRAFFIC/archive/yolov4.cfg'
name_file2 = 'D:/YOLOTRAFFIC/coco.names'
min_confidence = 0.4
min_confidence2 = 0.5

Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Other = [6, 12, 13, 14, 17, 32, 41, 42]

net = cv2.dnn.readNet(weight_file, cfg_file)
net2 = cv2.dnn.readNet(weight_file2, cfg_file2)

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
layer_names2 = net2.getLayerNames()
output_layers2 = [layer_names2[i[0] - 1] for i in net2.getUnconnectedOutLayers()]

classes = []
with open(name_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
classes2 = []
with open(name_file2, 'r') as f:
    classes2 = [line.strip() for line in f.readlines()]
    
model1 = tf.keras.models.load_model("C:/Users/HOME/ts_data_Prohibitory_aug1.h5", compile = True)
model2 = tf.keras.models.load_model("C:/Users/HOME/ts_data_Danger_aug1.h5", compile = True)
model3 = tf.keras.models.load_model("C:/Users/HOME/ts_data_Mandatory_aug1.h5", compile = True)
model4 = tf.keras.models.load_model("C:/Users/HOME/ts_data_Other_aug2.h5", compile = True)    

sign_information = [
    "Speed limit (20km/h)",
    "Speed limit (30km/h)",
    "Speed limit (50km/h)",
    "Speed limit (60km/h)",
    "Speed limit (70km/h)",
    "Speed limit (80km/h)",
    "End of speed limit (80km/h)",
    "Speed limit (100km/h)",
    "Speed limit (120km/h)",
    "No passing",
    "No passing for vechiles over 3.5 metric tons",
    "Right-of-way at the next intersection",
    "Priority road",
    "Yield",
    "Stop",
    "No vechiles",
    "Vechiles over 3.5 metric tons prohibited",
    "No entry",
    "General caution",
    "Dangerous curve to the left",
    "Dangerous curve to the right",
    "Double curve",
    "Bumpy road",
    "Slippery road",
    "Road narrows on the right",
    "Road work",
    "Traffic signals",
    "Pedestrians",
    "Children crossing",
    "Bicycles crossing",
    "Beware of ice/snow",
    "Wild animals crossing",
    "End of all speed and passing limits",
    "Turn right ahead",
    "Turn left ahead",
    "Ahead only",
    "Go straight or right",
    "Go straight or left",
    "Keep right",
    "Keep left",
    "Roundabout mandatory",
    "End of no passing",
    "End of no passing by vechiles over 3.5 metric tons"
]

In [8]:
random_number = 132
#file_name = 'D:/data_road_modi/dc.jpg'
file_name = 'D:/data_road_modi/images/0' + str(random_number) + '.jpg'
#file_name = 'C:/dd/awe.jpg'
#file_name = 'D:/maker_20210926/1dc4.jpg'
#file_name = 'D:/YOLOTRAFFIC/archive/home/my_name/ts/00' + str(random_number) + '.jpg'
#file_name = 'D:/YOLOTRAFFIC/archive/home/my_name/ts/00077.jpg'
a0 = time.time()
img = cv2.imread(file_name)
imgcopy = cv2.imread(file_name)
height, width, channels = img.shape
# Detecting objects
# https://docs.opencv.org/master/d6/d0f/group__dnn.html
blob = cv2.dnn.blobFromImage(img, 0.0392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []
box2  = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > min_confidence:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x, y, w, h])
            box2.append([center_x, center_y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
font = cv2.FONT_HERSHEY_COMPLEX
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        
        print(class_ids[i], label)
        
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #cv2.putText(img, label, (x, y - 10), font, 0.5, (0, 255, 0), 1)  
for i in range(len(box2)):
    if i in indexes:
        x2, y2, w2, h2 = box2[i]
        q2 = int(max(h2, w2) * 0.7)
        img2 = imgcopy[(y2-q2):(y2+q2), (x2-q2):(x2+q2)]
        
        
        dst = cv2.resize(img2, dsize=(32, 32), interpolation=cv2.INTER_AREA)
        dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
        if class_ids[i] == 0:
            aaqq = model1.predict(dst.reshape(1, 32, 32, 1))
            decode = np.argmax(aaqq, axis = 1)
            print(Prohibitory[decode[0]])
            print(sign_information[Prohibitory[decode[0]]])
            cv2.putText(img, sign_information[Prohibitory[decode[0]]], (x2 - w2 // 2, y2 - h2 // 2 - 10), font, 0.5, (0, 255, 0), 1)  
        elif class_ids[i] == 1:
            aaqq = model2.predict(dst.reshape(1, 32, 32, 1))
            decode = np.argmax(aaqq, axis = 1)
            print(Danger[decode[0]])
            print(sign_information[Danger[decode[0]]])
            cv2.putText(img, sign_information[Danger[decode[0]]], (x2 - w2 // 2, y2 - h2 // 2 - 10), font, 0.5, (0, 255, 0), 1)  
        elif class_ids[i] == 2:
            aaqq = model3.predict(dst.reshape(1, 32, 32, 1))
            decode = np.argmax(aaqq, axis = 1)
            print(Mandatory[decode[0]])
            print(sign_information[Mandatory[decode[0]]])
            cv2.putText(img, sign_information[Mandatory[decode[0]]], (x2 - w2 // 2, y2 - h2 // 2 - 10), font, 0.5, (0, 255, 0), 1)  
        elif class_ids[i] == 3:
            aaqq = model4.predict(dst.reshape(1, 32, 32, 1))
            decode = np.argmax(aaqq, axis = 1)
            print(Other[decode[0]])
            print(sign_information[Other[decode[0]]])
            cv2.putText(img, sign_information[Other[decode[0]]], (x2 - w2 // 2, y2 - h2 // 2 - 10), font, 0.5, (0, 255, 0), 1)  
            
net2.setInput(blob)
outs = net2.forward(output_layers2)

class_ids = []
confidences = []
boxes = []
box2  = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > min_confidence2:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x, y, w, h])
            box2.append([center_x, center_y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence2, 0.4)
font = cv2.FONT_HERSHEY_COMPLEX
for i in range(len(boxes)):
    if i in indexes and class_ids[i] in [0, 1, 2, 3, 5, 6, 7, 11, 12]:
        x, y, w, h = boxes[i]
        label = str(classes2[class_ids[i]])
        
        print(class_ids[i], label)
        
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(img, label, (x, y - 10), font, 0.5, (0, 0, 255), 1)  


cv2.imwrite('D:/R_20211023/' + str(random_number) + '_y26d.jpg', img)

a1 = time.time()
print(a1-a0)



0 prohibitory
2
Speed limit (50km/h)
5 bus
7 truck
7 truck
2 car
2 car
2 car
2 car
7 truck
0.795020341873169
